<h1 style="text-align: center;"><strong>PROJETO - ACESSO A API DO SPOTIFY</strong></h1>

<p>Este notebook serve como uma maneira pr&aacute;tica de entender e utilizar facilmente a API do Spotify. Ao seguir os passos, os leitores aprender&atilde;o como autenticar suas solicita&ccedil;&otilde;es, fazer chamadas &agrave; API e coletar dados brutos e transform&aacute;-los, tudo usando Python.</p>
<p>Nas se&ccedil;&otilde;es seguintes, o processo ser&aacute; dividido em etapas simples e acion&aacute;veis. Ao final, os leitores obter&atilde;o uma compreens&atilde;o abrangente da API do Spotify e a confian&ccedil;a para explorar outras APIs a fim de criar aplicativos variados e poderosos. Vamos come&ccedil;ar agora!</p>

<p><strong>Pr&eacute;-requisitos da API</strong></p>
<ul>
<li>Voc&ecirc; leu o <a href="https://developer.spotify.com/documentation/web-api/concepts/authorization">guia de autoriza&ccedil;&atilde;o</a>.</li>
<li>Voc&ecirc; criou um aplicativo seguindo o <a href="https://developer.spotify.com/documentation/web-api/concepts/apps">guia de aplicativos</a>.</li>
</ul>

# BIBLIOTECAS

In [1]:
import requests
import json
import base64
import os
import sqlite3
import datetime

import pandas as pd
import numpy  as np

from sqlalchemy      import create_engine
from IPython.display import display, HTML

## FUNÇÕES AUXILIARES

In [2]:
# aumentar o tamanho da célula do notebook (opcional)
display(HTML("<style>.container { width:75% !important; }</style>"))

## VARIÁVEIS DE AMBIENTE

<p>Por razões de segurança e privacidade, é aconselhável armazenar suas chaves como variáveis de ambiente para que, ao publicar seu caderno, não compartilhe dados sensíveis.</p>
<p><strong>Lembre-se de que ambas as chaves podem ser acessadas no painel do aplicativo Spotify se você seguiu o <a href="https://developer.spotify.com/documentation/web-api/concepts/apps">guia do app</a></strong></p>

In [4]:
# carregue as chaves
client_id = os.environ.get('SPOTIFY_CLIENT_ID')
client_secret = os.environ.get('SPOTIFY_CLIENT_SECRET')

In [5]:
# # load your client keys
# client_id = <your-client-id-key>
# client_secret = <your-client-secret-key>

# REQUISIÇÃO DO TOKEN DE ACESSO À API

In [6]:
# requisição a url da API
r = requests.post('https://accounts.spotify.com/api/token')

# Resposta 400 -> Requisição ruim - A requisição não pode ser entendida pelo servidor devido à sintaxe malformada
r

<Response [400]>

<p>Se a resposta recebida for 400, indica que a solicitação não atende aos parâmetros necessários para acessar a API. Nesses casos, é essencial consultar a documentação e fornecer o conjunto correto de informações. Especificamente para este projeto, consulte a documentação do <a href="https://developer.spotify.com/documentation/web-api/tutorials/client-credentials-flow">Client Credentials Flow</a> para os detalhes necessários.</p>
<p>Para solicitar autorização, siga estas etapas:</p>
<ul>
<li>Use o método POST com a URL base '<a href="https://accounts.spotify.com/" target="_new">https://accounts.spotify.com/</a>' e o endpoint '/api/token'.</li>
<li>Defina o parâmetro grant_type como 'client_credentials'.</li>
</ul>
<p>Para os parâmetros de cabeçalho, certifique-se do seguinte:</p>
<ul>
<li>A autorização deve estar no formato: Básico &lt;base64 codificado client_id:client_secret&gt;.</li>
<li>Defina o Content-type como 'application/x-www-form-urlencoded'.</li>
</ul>

In [7]:
# defina a URL base com o endpoint correto
url = 'https://accounts.spotify.com/api/token'

In [8]:
# utilize os parâmetros corretos
data = {'grant_type': 'client_credentials'}

In [9]:
# nossas chaves client_id e client_secret codificadas para base 64, conforme solicitado na documentação da api
base64_auth = base64.b64encode(f"{client_id}:{client_secret}".encode()).decode()

In [10]:
# forneça os headers corretos conforme indicado na documentação
headers = {'Authorization': 'Basic ' + base64_auth,
           'Content-Type' : 'application/x-www-form-urlencoded'}

In [11]:
# verifique se a api responde com o código 200
r = requests.post(url=url, headers=headers, data=data)

r # OK - a solicitação foi bem-sucedida

<Response [200]>

In [12]:
# verifique a resposta da API
#r.json()
r.json().keys()

dict_keys(['access_token', 'token_type', 'expires_in'])

## FUNÇÃO PARA OBTER DO TOKEN DE ACESSO

<p>Após testar com sucesso a acessibilidade da API do Spotify com as chaves fornecidas, torna-se mais fácil acessar usando uma função. Com base no exemplo presente na <a href="https://developer.spotify.com/documentation/web-api/tutorials/client-credentials-flow">documentação</a>, que é construído com JavaScript, desenvolve-se uma versão em Python com  uma descrição clara de seu uso e respostas para tentativas bem-sucedidas e falhas.</p>

In [13]:
def get_token(client_id, client_secret):
    """
    Obtém um token de acesso à API do Spotify usando credenciais de cliente e verifica seu tipo de token.

    Args:
        client_id (str): Chave ID do cliente da API do Spotify.
        client_secret (str): Chave Segredo do cliente da API do Spotify.

    Returns:
        str: Código de acesso obtido da API do Spotify.
        str: Tipo de token do código de acesso (ou None se falhar ao recuperar).
        str: Tempo de disponibilidade do token em segundos.
    """
    base64_auth = base64.b64encode(f"{client_id}:{client_secret}".encode()).decode()

    auth_options = {
        'url': 'https://accounts.spotify.com/api/token',
        'headers': {
            'Authorization': 'Basic ' + base64_auth, 
            'Content-Type' : 'application/x-www-form-urlencoded'
        },
        'data': {
            'grant_type': 'client_credentials'
        }
    }

    response = requests.post(auth_options['url'], headers=auth_options['headers'], data=auth_options['data'])

    if response.status_code == 200:
        r = response.json()
        token = r['access_token']
        token_type = r['token_type']
        token_duration = r['expires_in']
        print(f'Token de Acesso requisitado com successo!')
        print(f'Tipo do Token: {token_type}')
        print(f'Disponibilidade do Token: {token_duration} segundos')
    else:
        print('Não foi possível obter o token de acesso')
    
    return f'{token_type} {token}'

In [14]:
# teste da função
access_token = get_token(client_id, client_secret)

Token de Acesso requisitado com successo!
Tipo do Token: Bearer
Disponibilidade do Token: 3600 segundos


# REQUISITANDO OS DADOS DA API

<p>Para iniciar nossa solicitação, vamos começar selecionando três gêneros específicos: Blues, Rock e Soul. Antes de prosseguir, é necessário consultar a documentação para identificar os endpoints corretos dos dados que pretendemos acessar. Para este projeto, nosso foco está nos gêneros e suas respectivas faixas.</p>
<p>Primeiro, consulte a seção <a href="https://developer.spotify.com/documentation/web-api/reference/get-recommendation-genres">Recommendation Genres</a> na documentação para confirmar a disponibilidade dos gêneros escolhidos: Blues, Rock e Soul. Uma vez verificado, acesse a URL especificada para recuperar os dados de origem desses gêneros.</p>

## GÊNEROS DISPONÍVEIS

In [15]:
# url com os gêneros disponíveis.
url = 'https://api.spotify.com/v1/recommendations/available-genre-seeds'

In [16]:
# se a requisição estiver correta, ela deve retornar o código 200
response = requests.get(url, headers={'Authorization': access_token})
response

<Response [200]>

In [17]:
# verifique o texto ou JSON da resposta
response.text

'{\n  "genres" : [ "acoustic", "afrobeat", "alt-rock", "alternative", "ambient", "anime", "black-metal", "bluegrass", "blues", "bossanova", "brazil", "breakbeat", "british", "cantopop", "chicago-house", "children", "chill", "classical", "club", "comedy", "country", "dance", "dancehall", "death-metal", "deep-house", "detroit-techno", "disco", "disney", "drum-and-bass", "dub", "dubstep", "edm", "electro", "electronic", "emo", "folk", "forro", "french", "funk", "garage", "german", "gospel", "goth", "grindcore", "groove", "grunge", "guitar", "happy", "hard-rock", "hardcore", "hardstyle", "heavy-metal", "hip-hop", "holidays", "honky-tonk", "house", "idm", "indian", "indie", "indie-pop", "industrial", "iranian", "j-dance", "j-idol", "j-pop", "j-rock", "jazz", "k-pop", "kids", "latin", "latino", "malay", "mandopop", "metal", "metal-misc", "metalcore", "minimal-techno", "movies", "mpb", "new-age", "new-release", "opera", "pagode", "party", "philippines-opm", "piano", "pop", "pop-film", "post-d

In [18]:
# verifique as chaves de resposta
response.json().keys()

dict_keys(['genres'])

**É engraçado ver que o Brasil e a Disney têm seu próprio gênero.**

In [19]:
# crie uma lista com todos os gêneros
spotify_genres = response.json()['genres']
len(spotify_genres)

126

In [20]:
# os gêneros de exemplo para este projeto em minúsculas
example_genres = ['blues', 'rock', 'soul']

selected_genres = []

# verificando se nossos gêneros estão disponíveis na lista de gênero
for genre in example_genres: 
    if genre in spotify_genres: 
        selected_genres.append(genre)
        
selected_genres

['blues', 'rock', 'soul']

## DADOS DAS FAIXAS

<p>Após confirmar os gêneros na lista, vamos explorar os dados de cada faixa. Neste caso, a escolha mais adequada é a <a href="https://developer.spotify.com/documentation/web-api/reference/search">Search for Item Reference</a>. Este método permite filtrar por gênero e possibilita solicitações de informações relacionadas a álbuns, artistas, playlists, faixas, shows, episódios ou audiobooks.</p>
<p>Embora seja possível usar a opção <a href="https://developer.spotify.com/documentation/web-api/reference/get-several-tracks">Get Several Tracks</a> da API, isso requer fornecer IDs de faixas, com um máximo de 50 IDs por lote. No entanto, isso não é possível no momento, pois ainda não temos acesso aos IDs das faixas.</p>

<p>No <a href="https://developer.spotify.com/documentation/web-api/reference/search">Search for Item Reference</a>, podemos observar alguns dos parâmetros de solicitação. Aqui está um resumo de cada um:</p>
<ul>
<li><strong>q (string) [Obrigatório]</strong>: <br />Sua consulta de pesquisa. Pode ser refinada usando filtros de campo, incluindo álbum, artista, faixa, ano, upc, tag:hipster, tag:new, isrc e gênero. Cada filtro se aplica a tipos de resultados específicos.<br />- Filtros de artista e ano se aplicam a álbuns, artistas e faixas.<br />- O filtro de álbum se aplica a álbuns e faixas.<br />- O filtro de gênero se aplica a artistas e faixas.<br />- Os filtros ISRC e de faixa se aplicam a faixas.<br />- Os filtros UPC, tag:new e tag:hipster são para álbuns. tag:new retorna álbuns das últimas duas semanas, e tag:hipster filtra os álbuns com menor popularidade, abaixo de 10%.<br />Exemplo de valor: "remaster%20track:Doxy%20artist:Miles%20Davis"</li>
<li><strong>type (array de strings) [Obrigatório]</strong>: <br />Lista de tipos de itens separados por vírgulas para pesquisar. Os resultados incluem correspondências de todos os tipos de itens especificados: "album", "artist", "playlist", "track", "show", "episode", "audiobook".<br />Exemplo: type=album,track retorna tanto álbuns quanto faixas que correspondem à consulta.</li>
<li><strong>market (string)</strong>: <br />Código do país no formato ISO 3166-1 alpha-2. Especifica o mercado para disponibilidade de conteúdo. Se um token de acesso do usuário for fornecido, o país associado ao usuário tem prioridade. Se nem o mercado nem o país do usuário forem fornecidos, o conteúdo é considerado indisponível para o cliente.<br />Exemplo de valor: "BR"</li>
<li><strong>limit (integer)</strong>: <br />Número máximo de resultados a serem retornados para cada tipo de item.<br />Exemplo de valor: 10<br />Valor padrão: 20<br />Faixa: 0 - 50</li>
<li><strong>offset (integer)</strong>: <br />Índice do primeiro resultado a ser retornado. Usado com o limit para paginar pelos resultados da pesquisa.<br />Exemplo de valor: 5<br />Valor padrão: 0<br />Faixa: 0 - 1000</li>
<li><strong>include_external (string)</strong>: <br />Se include_external=audio for especificado, indica que o cliente pode reproduzir conteúdo de áudio hospedado externamente. Por padrão, o conteúdo de áudio hospedado externamente é marcado como não reproduzível na resposta. Valor permitido: "audio".</li>
</ul>

<p>Para este projeto, foram selecionados os seguintes parâmetros:</p>
<ul>
<li><strong>q</strong> = genre</li>
<li><strong>type</strong> = track</li>
<li><strong>market</strong> = BR</li>
<li><strong>limit</strong> = 20</li>
<li><strong>offset</strong> = 0</li>
</ul>

In [21]:
# nosso endpoint com base na documentação - exemplo.
url = 'https://api.spotify.com/v1/search?q=genre:rock&type=track&market=BR&limit=20&offset=0'

In [22]:
# se a requisição estiver correta, ela deve retornar o código 200
response = requests.get(url, headers={'Authorization': access_token})
response

<Response [200]>

In [23]:
# verifique os dados na resposta completa
response.json()

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=genre%3Arock&type=track&market=BR&offset=0&limit=20',
  'items': [{'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/7Ln80lUS6He07XvHI8qqHH'},
       'href': 'https://api.spotify.com/v1/artists/7Ln80lUS6He07XvHI8qqHH',
       'id': '7Ln80lUS6He07XvHI8qqHH',
       'name': 'Arctic Monkeys',
       'type': 'artist',
       'uri': 'spotify:artist:7Ln80lUS6He07XvHI8qqHH'}],
     'external_urls': {'spotify': 'https://open.spotify.com/album/78bpIziExqiI9qztvNFlQu'},
     'href': 'https://api.spotify.com/v1/albums/78bpIziExqiI9qztvNFlQu',
     'id': '78bpIziExqiI9qztvNFlQu',
     'images': [{'height': 640,
       'url': 'https://i.scdn.co/image/ab67616d0000b2734ae1c4c5c45aabe565499163',
       'width': 640},
      {'height': 300,
       'url': 'https://i.scdn.co/image/ab67616d00001e024ae1c4c5c45aabe565499163',
       'width': 300},
      {'height': 64,
       'url': 'htt

In [24]:
# as chaves obtidas da resposta da API
response.json().keys()

dict_keys(['tracks'])

In [25]:
# verifique os dados na resposta completa da chave tracks
response.json()['tracks']

{'href': 'https://api.spotify.com/v1/search?query=genre%3Arock&type=track&market=BR&offset=0&limit=20',
 'items': [{'album': {'album_type': 'album',
    'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/7Ln80lUS6He07XvHI8qqHH'},
      'href': 'https://api.spotify.com/v1/artists/7Ln80lUS6He07XvHI8qqHH',
      'id': '7Ln80lUS6He07XvHI8qqHH',
      'name': 'Arctic Monkeys',
      'type': 'artist',
      'uri': 'spotify:artist:7Ln80lUS6He07XvHI8qqHH'}],
    'external_urls': {'spotify': 'https://open.spotify.com/album/78bpIziExqiI9qztvNFlQu'},
    'href': 'https://api.spotify.com/v1/albums/78bpIziExqiI9qztvNFlQu',
    'id': '78bpIziExqiI9qztvNFlQu',
    'images': [{'height': 640,
      'url': 'https://i.scdn.co/image/ab67616d0000b2734ae1c4c5c45aabe565499163',
      'width': 640},
     {'height': 300,
      'url': 'https://i.scdn.co/image/ab67616d00001e024ae1c4c5c45aabe565499163',
      'width': 300},
     {'height': 64,
      'url': 'https://i.scdn.co/image/ab67616d

In [26]:
# as chaves que podemos acessar a partir da resposta das faixas (tracks)
response.json()['tracks'].keys()

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])

<p>Dê uma olhada dentro da chave 'tracks' e você encontrará várias outras chaves, cada uma com diferentes valores de dados. Ao consultar a <a href="https://developer.spotify.com/documentation/web-api/reference/search">documentação</a> e a resposta, é fácil ver que a chave 'items' contém os dados relacionados às faixas e quais dados serão selecionados.</p>

In [27]:
# as chaves que podemos acessar a partir da resposta das faixas tracks e items
response.json()['tracks']['items'][0].keys()

dict_keys(['album', 'artists', 'disc_number', 'duration_ms', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'is_playable', 'name', 'popularity', 'preview_url', 'track_number', 'type', 'uri'])

In [28]:
# primeiro item na chave items
response.json()['tracks']['items'][0]

{'album': {'album_type': 'album',
  'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/7Ln80lUS6He07XvHI8qqHH'},
    'href': 'https://api.spotify.com/v1/artists/7Ln80lUS6He07XvHI8qqHH',
    'id': '7Ln80lUS6He07XvHI8qqHH',
    'name': 'Arctic Monkeys',
    'type': 'artist',
    'uri': 'spotify:artist:7Ln80lUS6He07XvHI8qqHH'}],
  'external_urls': {'spotify': 'https://open.spotify.com/album/78bpIziExqiI9qztvNFlQu'},
  'href': 'https://api.spotify.com/v1/albums/78bpIziExqiI9qztvNFlQu',
  'id': '78bpIziExqiI9qztvNFlQu',
  'images': [{'height': 640,
    'url': 'https://i.scdn.co/image/ab67616d0000b2734ae1c4c5c45aabe565499163',
    'width': 640},
   {'height': 300,
    'url': 'https://i.scdn.co/image/ab67616d00001e024ae1c4c5c45aabe565499163',
    'width': 300},
   {'height': 64,
    'url': 'https://i.scdn.co/image/ab67616d000048514ae1c4c5c45aabe565499163',
    'width': 64}],
  'is_playable': True,
  'name': 'AM',
  'release_date': '2013-09-09',
  'release_date_precisio

<p>Há muitos dados que podemos extrair daqui, os selecionados:</p>
<ul>
<li>Track ID</li>
<li>Track Name</li>
<li>Track Duration</li>
<li>Artist Name</li>
<li>Album Name</li>
<li>Album Release Date</li>
<li>Popularity</li>
</ul>
<p>Vamos ver como podemos acessar cada um.</p>

In [29]:
# id da faixa do primeiro item
response.json()['tracks']['items'][0]['id']

'5XeFesFbtLpXzIVDNQP22n'

In [30]:
# nome da faixa do primeiro item
response.json()['tracks']['items'][0]['name']

'I Wanna Be Yours'

In [31]:
# duração da faixa em milissegundos do primeiro item.
response.json()['tracks']['items'][0]['duration_ms']

183956

In [32]:
# nome do artista do primeiro item
response.json()['tracks']['items'][0]['artists'][0]['name']

'Arctic Monkeys'

In [33]:
# nome do álbum do primeiro item
response.json()['tracks']['items'][0]['album']['name']

'AM'

In [34]:
# data de lançamento do álbum do primeiro item
response.json()['tracks']['items'][0]['album']['release_date']

'2013-09-09'

In [35]:
# popularidade do primeiro item
response.json()['tracks']['items'][0]['popularity']

95

In [36]:
track_id = response.json()['tracks']['items'][0]['id']

In [37]:
# cada característica que selecionamos na resposta da API será atribuída a uma variável.
track_id = response.json()['tracks']['items'][0]['id']
track_name = response.json()['tracks']['items'][0]['name']
track_duration_ms = response.json()['tracks']['items'][0]['duration_ms']
artist_name = response.json()['tracks']['items'][0]['artists'][0]['name']
album_name = response.json()['tracks']['items'][0]['album']['name']
album_release_date = response.json()['tracks']['items'][0]['album']['release_date']
popularity = response.json()['tracks']['items'][0]['popularity']
genre = 'rock' 

# crie um DataFrame para visualizar os dados
track_df = pd.DataFrame(
    {
    'track_id': [track_id],
    'track_name': [track_name],
    'track_duration_ms':[track_duration_ms],
    'artist_name': [artist_name],
    'album_name': [album_name],
    'album_release_date': [album_release_date],
    'popularity': [popularity],
    'genre': [genre]
    }
    )

In [38]:
print(f'Este DataFrame possui {track_df.shape[1]} colunas')
track_df

Este DataFrame possui 8 colunas


,track_id,track_name,track_duration_ms,artist_name,album_name,album_release_date,popularity,genre
0,5XeFesFbtLpXzIVDNQP22n,I Wanna Be Yours,183956,Arctic Monkeys,AM,2013-09-09,95,rock


## FUNÇÃO PARA COLETA DE DADOS

<p>Agora, tudo o que precisamos é de uma função para acessar a API e coletar os dados. Assim como fizemos anteriormente, esta função incluirá uma descrição clara de seu uso e respostas.</p>
<p><strong>**Para este código, vou construí-lo passo a passo apenas para compartilhar como foi abordado e meu processo de pensamento durante sua construção**</strong></p>


<p><em>Obs: Como aprendi do jeito difícil, é melhor verificar primeiro como a URL é construída para entender como as informações precisam ser fornecidas. Isso garante que os parâmetros sejam fornecidos na ordem correta.</em></p>

### PASSO 1 - API CALL

<p>Vamos transformar o código que foi construído para chamar a API em uma função. Como será usado com parâmetros que determinamos, é necessário generalizar a URL. Quando chamada, os parâmetros serão fornecidos.</p>

In [39]:
def api_call(url, access_token):
    """
    Chama a API do Spotify usando um endpoint de URL e um token de acesso para recuperar dados no formato JSON.

    Args:
        url (str): Endpoint de URL para acessar dados de faixas disponíveis.
            URL padrão fornecida - https://api.spotify.com/v1/search?q=genre:{genre}&type=track&market={market}&limit={limit}&offset={offset}
        
        access_token (str): Token de acesso à API do Spotify fornecido com o uso da função get_token.

    Returns:
        dict: Objeto JSON com a resposta da API.
    """
    response = requests.get(url, headers={'Authorization': access_token})
    api_response = response.json()
    
    return api_response

In [40]:
# exemplo
market = 'BR'
genre = 'soul'
limit = 5
offset = 0

# definindo nosso endpoint com base na documentação
url = f'https://api.spotify.com/v1/search?q=genre:{genre}&type=track&market={market}&limit={limit}&offset={offset}'

# se a requisição estiver correta, ela deve imprimir o JSON
api_call(url, access_token)

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=genre%3Asoul&type=track&market=BR&offset=0&limit=5',
  'items': [{'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/4dpARuHxo51G3z768sgnrY'},
       'href': 'https://api.spotify.com/v1/artists/4dpARuHxo51G3z768sgnrY',
       'id': '4dpARuHxo51G3z768sgnrY',
       'name': 'Adele',
       'type': 'artist',
       'uri': 'spotify:artist:4dpARuHxo51G3z768sgnrY'}],
     'external_urls': {'spotify': 'https://open.spotify.com/album/0Lg1uZvI312TPqxNWShFXL'},
     'href': 'https://api.spotify.com/v1/albums/0Lg1uZvI312TPqxNWShFXL',
     'id': '0Lg1uZvI312TPqxNWShFXL',
     'images': [{'height': 640,
       'url': 'https://i.scdn.co/image/ab67616d0000b2732118bf9b198b05a95ded6300',
       'width': 640},
      {'height': 300,
       'url': 'https://i.scdn.co/image/ab67616d00001e022118bf9b198b05a95ded6300',
       'width': 300},
      {'height': 64,
       'url': 'https://i.scd

In [41]:
# salvando a resposta em uma variável
api_response = api_call(url, access_token)

In [42]:
# verificando o comprimento da resposta para corresponder ao limite - OK
len(api_response['tracks']['items'])

5

## PASSO 2 - ITERAR NÚMERO DA FAIXA, GÊNERO, LIMITE E OFFSET

<p>O processo de construção foi iniciado usando o código desenvolvido para criar o DataFrame. Como os dados serão coletados de diferentes faixas, é vital criar um método generalizado para que nosso código possa iterar.</p>
<p>Para alcançar isso, alteramos [0] observado após ['items'] e ['tracks'] para [i], pois corresponde ao número de cada faixa na resposta. Uma lista determinará o gênero, referido como 'gênero', e o limite ditará quantas vezes o código iterará por página.</p>
<p>Além disso, a funcionalidade response.json() foi alterada para api_response para corresponder ao objeto de retorno de nossa função api_call.</p>

In [43]:
# exemplo
selected_genres = ['blues', 'rock', 'soul']
limit = 5
offset = 0

# definindo nosso endpoint com base na documentação
url = f'https://api.spotify.com/v1/search?q=genre:{genre}&type=track&market=BR&limit={limit}&offset={offset}'

# se a requisição estiver correta, ela deve retornar o JSON
api_response = api_call(url, access_token)

In [44]:
for genre in selected_genres:

    for i in range(limit):
        track_id = api_response['tracks']['items'][i]['id']
        track_name = api_response['tracks']['items'][i]['name']
        track_duration_ms = api_response['tracks']['items'][i]['duration_ms']
        artist_name = api_response['tracks']['items'][i]['artists'][0]['name']
        album_name = api_response['tracks']['items'][i]['album']['name']
        album_release_date = api_response['tracks']['items'][i]['album']['release_date']
        popularity = api_response['tracks']['items'][i]['popularity']
        genre = genre  # como informamos na solicitação de URL no início desta seção
        # criar um novo DataFrame com os dados
        track_df = pd.DataFrame([
            {
            'track_id': track_id,
            'track_name': track_name,
            'track_duration_ms':track_duration_ms,
            'artist_name': artist_name,
            'album_name': album_name,
            'album_release_date': album_release_date,
            'popularity': popularity,
            'genre': genre
            }], index=None
            )

In [45]:
track_df

,track_id,track_name,track_duration_ms,artist_name,album_name,album_release_date,popularity,genre
0,1zwMYTA5nlNjZxYrvBB2pV,Someone Like You,285240,Adele,21,2011-01-24,81,soul


<p>O código está retornando apenas uma linha porque não está adicionando mais valores ao DataFrame. Para corrigir isso, pode ser mais fácil preencher uma lista vazia com objetos de dicionário e depois convertê-la em um DataFrame. Além disso, o código não está iterando pela lista de gêneros, provavelmente porque o parâmetro de gênero da URL está definido fora do loop.</p>

In [46]:
selected_genres = ['blues', 'rock', 'soul']
limit = 5
offset = 0
track_df = []

for genre in selected_genres:
    url = f'https://api.spotify.com/v1/search?q=genre:{genre}&type=track&market=BR&limit={limit}&offset={offset}'
    api_response = api_call(url, access_token)
    
    for i in range(limit):
        track_id = api_response['tracks']['items'][i]['id']
        track_name = api_response['tracks']['items'][i]['name']
        track_duration_ms = api_response['tracks']['items'][i]['duration_ms']
        artist_name = api_response['tracks']['items'][i]['artists'][0]['name']
        album_name = api_response['tracks']['items'][i]['album']['name']
        album_release_date = api_response['tracks']['items'][i]['album']['release_date']
        popularity = api_response['tracks']['items'][i]['popularity']
        genre = genre

        track_df.append(
            {
            'track_id': track_id,
            'track_name': track_name,
            'track_duration_ms':track_duration_ms,
            'artist_name': artist_name,
            'album_name': album_name,
            'album_release_date': album_release_date,
            'popularity': popularity,
            'genre': genre
            }
            )
            
tracks_dataset = pd.DataFrame(track_df)

In [47]:
tracks_dataset

,track_id,track_name,track_duration_ms,artist_name,album_name,album_release_date,popularity,genre
0,3dPQuX8Gs42Y7b454ybpMR,Seven Nation Army,232106,The White Stripes,Elephant,2003-04-01,87,blues
1,63OFKbMaZSDZ4wtesuuq6f,Born To Be Wild,210373,Steppenwolf,Steppenwolf,1968,76,blues
2,5G1sTBGbZT5o4PNRc75RKI,Lonely Boy,193653,The Black Keys,El Camino,2011-12-06,80,blues
3,5MAK1nd8R6PWnle1Q1WJvh,I See Red,230613,Everybody Loves an Outlaw,I See Red,2018-10-31,77,blues
4,6jHvX8ZnHKC1PnrPMJ0Emt,Cigarette Daydreams,208760,Cage The Elephant,Melophobia,2013-10-08,77,blues
5,5XeFesFbtLpXzIVDNQP22n,I Wanna Be Yours,183956,Arctic Monkeys,AM,2013-09-09,95,rock
6,006pMMCuRo2TFjh8sNGSov,"Dias De Luta, Dias De Gloria",145466,Charlie Brown Jr.,Imunidade Musical,2005-01-01,75,rock
7,3nI0piSOxAik2RCpHGloB7,Só os Loucos Sabem,210493,Charlie Brown Jr.,Camisa 10 joga bola até na chuva,2009-09-16,75,rock
8,2QjOHCTQ1Jl3zawyYOpxh6,Sweater Weather,240400,The Neighbourhood,I Love You.,2013-04-19,93,rock
9,7MnT7msJZg3XBAS0OTfGrB,Tempo Perdido,302200,Legião Urbana,Dois,1986-01-01,73,rock


<p>Para usar a API de forma eficaz, é importante ajustar corretamente o parâmetro de deslocamento (offset). Este parâmetro trabalha com o valor do limite (limit) para passar para a próxima página de dados. Para cada gênero, o parâmetro de deslocamento precisa ser resetado para o seu valor original para que possa ser iterado novamente. Para acompanhar isso, a variável "pages" foi adicionada. Ela ajuda a saber quantas vezes a função precisa ser percorrida.</p>
<p><span style="text-decoration: underline;"><em>Para extrair o máximo da API, é crucial consultar a documentação. A API permite no máximo 50 faixas por página, e o valor máximo para o parâmetro de deslocamento é 1000. Portanto, para cada gênero, podemos extrair 20 páginas, cada uma contendo 50 músicas (20 x 50 = 1000). O maior conjunto de dados que podemos teoricamente obter, considerando nosso código, terá 126.000 faixas (126 gêneros x limite de 1000 faixas).</em></span></p>


In [48]:
selected_genres = ['blues', 'rock', 'soul']
limit = 50
offset = 0
pages = 20
offset_counter = offset
track_df = []

for genre in selected_genres:
    
    for page in range(pages):    
        url = f'https://api.spotify.com/v1/search?q=genre:{genre}&type=track&market=BR&limit={limit}&offset={offset}'
        api_response = api_call(url, access_token)
        
        for i in range(limit):
            track_id = api_response['tracks']['items'][i]['id']
            track_name = api_response['tracks']['items'][i]['name']
            track_duration_ms = api_response['tracks']['items'][i]['duration_ms']
            artist_name = api_response['tracks']['items'][i]['artists'][0]['name']
            album_name = api_response['tracks']['items'][i]['album']['name']
            album_release_date = api_response['tracks']['items'][i]['album']['release_date']
            popularity = api_response['tracks']['items'][i]['popularity']
            genre = genre

            # append data to the empty list
            track_df.append(
                {
                'track_id': track_id,
                'track_name': track_name,
                'track_duration_ms':track_duration_ms,
                'artist_name': artist_name,
                'album_name': album_name,
                'album_release_date': album_release_date,
                'popularity': popularity,
                'genre': genre
                }
                )
        offset += limit
    offset = offset_counter
tracks_dataset = pd.DataFrame(track_df)

In [49]:
print(f'Este dataset possui {tracks_dataset.shape[0]} linhas e {tracks_dataset.shape[1]} colunas')
tracks_dataset

Este dataset possui 3000 linhas e 8 colunas


,track_id,track_name,track_duration_ms,artist_name,album_name,album_release_date,popularity,genre
0,3dPQuX8Gs42Y7b454ybpMR,Seven Nation Army,232106,The White Stripes,Elephant,2003-04-01,87,blues
1,63OFKbMaZSDZ4wtesuuq6f,Born To Be Wild,210373,Steppenwolf,Steppenwolf,1968,76,blues
2,5G1sTBGbZT5o4PNRc75RKI,Lonely Boy,193653,The Black Keys,El Camino,2011-12-06,80,blues
3,5MAK1nd8R6PWnle1Q1WJvh,I See Red,230613,Everybody Loves an Outlaw,I See Red,2018-10-31,77,blues
4,6jHvX8ZnHKC1PnrPMJ0Emt,Cigarette Daydreams,208760,Cage The Elephant,Melophobia,2013-10-08,77,blues
...,...,...,...,...,...,...,...,...
2995,6hpmTwgNCz81H2bFEREx29,"Get Up I Feel Like Being Like A Sex Machine, P...",318800,James Brown,70's Funk Classics,1995-04-16,58,soul
2996,6T2nouxfQX7kFHUCsWiPJk,Candy,298693,Paolo Nutini,Musica per innamorarsi,2023-10-19,0,soul
2997,0DC6yJLAPwIEeZh6EZpn1f,So Done (feat. Khalid),234866,Alicia Keys,ALICIA,2020-12-18,54,soul
2998,6fQvFdM1wUqndgu2fw6CcO,Last Request,220413,Paolo Nutini,Mañanita Chill,2023-10-19,0,soul


### PASSO 3 - CONSOLIDAR EM UMA FUNÇÃO

<p>Com tudo configurado, só precisamos generalizar e marcar cada variável necessária para chamar nossa função.</p>

In [50]:
def tracks_dataset(genres, limit, offset, pages, access_token):

    offset_counter = offset
    track_df = []

    for genre in genres: 
        
        for page in range(pages):    
            url = f'https://api.spotify.com/v1/search?q=genre:{genre}&type=track&market=BR&limit={limit}&offset={offset}'
            api_response = api_call(url, access_token)
            
            for i in range(limit):
                track_id = api_response['tracks']['items'][i]['id']
                track_name = api_response['tracks']['items'][i]['name']
                track_duration_ms = api_response['tracks']['items'][i]['duration_ms']
                artist_name = api_response['tracks']['items'][i]['artists'][0]['name']
                album_name = api_response['tracks']['items'][i]['album']['name']
                album_release_date = api_response['tracks']['items'][i]['album']['release_date']
                popularity = api_response['tracks']['items'][i]['popularity']
                genre = genre 

                track_df.append(
                    {
                    'track_id': track_id,
                    'track_name': track_name,
                    'track_duration_ms':track_duration_ms,
                    'artist_name': artist_name,
                    'album_name': album_name,
                    'album_release_date': album_release_date,
                    'popularity': popularity,
                    'genre': genre
                    }
                    )
            offset += limit
        offset = offset_counter
    
    tracks_dataset = pd.DataFrame(track_df, index = None)    
    
    return tracks_dataset

In [51]:
# exemplo com os gêneros selecionados
genres = ['alt-rock', 'alternative', 'brazil', 'blues', 'electro', 'heavy-metal', 'hip-hop', 'house', 'jazz', 'pop', 'reggae', 'rock', 'soul', 'techno', 'trance'] 

limit = 50
offset = 0
pages = 20
access_token = access_token

In [52]:
tracks_dataset(genres, limit, offset, pages, access_token)

,track_id,track_name,track_duration_ms,artist_name,album_name,album_release_date,popularity,genre
0,3nI0piSOxAik2RCpHGloB7,Só os Loucos Sabem,210493,Charlie Brown Jr.,Camisa 10 joga bola até na chuva,2009-09-16,75,alt-rock
1,2QjOHCTQ1Jl3zawyYOpxh6,Sweater Weather,240400,The Neighbourhood,I Love You.,2013-04-19,93,alt-rock
2,70dJEanFPdYuWZumkrnKeX,Ela Vai Voltar (Todos Os Defeitos De Uma Mulhe...,188200,Charlie Brown Jr.,Imunidade Musical,2005-01-01,73,alt-rock
3,1z1EeTXwnz3gvoUvzvkkdw,Céu Azul - Ao Vivo,200071,Charlie Brown Jr.,Música Popular Caiçara: Edição Luxo (Ao Vivo),2012-04-12,72,alt-rock
4,31AOj9sFz2gM0O3hMARRBx,Losing My Religion,268426,R.E.M.,Out Of Time (25th Anniversary Edition),1991-03-12,87,alt-rock
...,...,...,...,...,...,...,...,...
14995,403CnUCf17AVH38yLQQktC,Sometimes We Cry,363428,Vegas (Brazil),Tempoo,2022-02-22,27,trance
14996,0O7fXel2uzgJraKd1NBSVb,Beat It,210639,One Morning Left,Beat It,2021-10-29,52,trance
14997,6n2OgfyvY0MuwFphEywfgz,Ancient Aum,480000,Electric Universe,Ancient Aum,2022-09-02,42,trance
14998,3vOPqbFvgs7Xjt0KWct1vt,Jericho - Remix,244965,Claudinho Brasil,Jericho (Remix),2023-06-02,35,trance


<p>Um erro ocorreu durante uma tentativa de solicitar todos os dados disponíveis em vários gêneros. Após investigar, foi descoberto que o índice de um item na resposta da API é perdido quando um gênero possui faixas insuficientes para o valor máximo solicitado. Para resolver esse problema, foi implementado um comando "break" no código quando o número de itens é abaixo do limite ou igual a zero. Isso garante que cada lote de dados contenha 50 faixas. Além disso, nesta última interação da função, a variável "markets" foi adicionada.</p>
<p><span style="text-decoration: underline;"><em>Obs.: Se múltiplos mercados são informados, isso leva a um aumento no número de solicitações à API. Se essas solicitações excederem o limite de tempo de disponibilidade do token (apenas 1 hora), o código será interrompido.</em></span></p>

In [53]:
def tracks_dataset(genres, markets, limit, offset, pages, access_token):
    """
    Recupera dados de faixas da API do Spotify com base nos gêneros especificados, parâmetros de paginação e token de acesso.

    Args:
        genres (list): Uma lista de nomes de gêneros para os quais as faixas precisam ser obtidas da API do Spotify.
        markets (list): Uma lista de códigos de país para os mercados disponíveis no Spotify (valor de acordo com o código de país ISO 3166-1 alpha-2).
        limit (int): O número máximo de faixas a serem obtidas por requisição à API.
        offset (int): O offset inicial para paginação para obter faixas da API.
        pages (int): O número de páginas da API a serem buscadas para cada gênero.
        access_token (str): O token de acesso necessário para autenticação com a API do Spotify.

    Returns:
        pandas.DataFrame: Um DataFrame do pandas contendo as informações das faixas extraídas, incluindo ID da faixa,
        nome da faixa, duração, nome do artista, nome do álbum, data de lançamento do álbum, popularidade e gênero.

    Funcionalidades:
        - Itera pelos gêneros e páginas de paginação especificados para obter dados de faixas da API do Spotify.
        - Extrai informações relevantes da resposta da API.
        - Combina os dados extraídos em um DataFrame do pandas e retorna o DataFrame ao chamador.

    Observações:
        - A função faz várias chamadas à API com base nos gêneros e na paginação, processando os dados em um DataFrame.
        - Usa o formato de URL do endpoint da API fornecido com espaços reservados para gênero, limite e offset.
        - Assume a existência de uma função 'api_call' para fazer requisições à API.
        - Requer o manuseio adequado do token de acesso usando a função 'get_token' ou um mecanismo semelhante.
    """

    offset_counter = offset
    track_df = []
    
    for market in markets:
        for genre in genres:  
            for page in range(pages):    
                url = f'https://api.spotify.com/v1/search?q=genre:{genre}&type=track&market={market}&limit={limit}&offset={offset}'
                api_response = api_call(url, access_token)
                num_items = len(api_response['tracks']['items'])

                if num_items == 0:
                    break

                for i in range(num_items):
                    track_id = api_response['tracks']['items'][i]['id']
                    track_name = api_response['tracks']['items'][i]['name']
                    track_duration_ms = api_response['tracks']['items'][i]['duration_ms']
                    artist_name = api_response['tracks']['items'][i]['artists'][0]['name']
                    album_name = api_response['tracks']['items'][i]['album']['name']
                    album_release_date = api_response['tracks']['items'][i]['album']['release_date']
                    popularity = api_response['tracks']['items'][i]['popularity']
                    genre = genre
                    market = market

                    track_df.append({
                        'track_id': track_id,
                        'track_name': track_name,
                        'track_duration_ms': track_duration_ms,
                        'artist_name': artist_name,
                        'album_name': album_name,
                        'album_release_date': album_release_date,
                        'popularity': popularity,
                        'genre': genre,
                        'market': market
                    })

                offset += limit

                if num_items < limit:
                    break

            offset = offset_counter

        tracks_dataset = pd.DataFrame(track_df)    
        return tracks_dataset

In [54]:
# exemplo com os todos os gêneros
genres = spotify_genres 
markets = ['BR']
limit = 50
offset = 0
pages = 20
access_token = access_token

In [55]:
tracks_df = tracks_dataset(genres, markets, limit, offset, pages, access_token)

In [56]:
print(f'Este dataset possui {tracks_df.shape[0]} linhas e {tracks_df.shape[1]} colunas')
tracks_df.head()

Este dataset possui 112400 linhas e 9 colunas


,track_id,track_name,track_duration_ms,artist_name,album_name,album_release_date,popularity,genre,market
0,38jy6kRlPt8z1GUS9WXeNh,93 Million Miles,216386,Jason Mraz,Love Is a Four Letter Word,2012-04-13,69,acoustic,BR
1,3S0OXQeoh0w6AY8WQVckRW,I'm Yours,242946,Jason Mraz,We Sing. We Dance. We Steal Things. (Bonus Tra...,2008-05-01,79,acoustic,BR
2,5vjLSffimiIP26QG5WcN2K,Hold On,198853,Chord Overstreet,Hold On,2017-02-03,83,acoustic,BR
3,1EzrEOXmMH3G43AXT1y7pA,I'm Yours,242946,Jason Mraz,We Sing. We Dance. We Steal Things.,2008-05-12,81,acoustic,BR
4,2qLMf6TuEC3ruGJg4SMMN6,Lucky,189613,Jason Mraz,We Sing. We Dance. We Steal Things. (Bonus Tra...,2008-05-01,74,acoustic,BR


## EXPORTAR PARA CSV E SQLITE

<p>O DataFrame criado pode ser exportado para um arquivo CSV para facilitar o carregamento ou usado para analisar e armazenar dados em um arquivo SQLite.</p>

### CSV

In [57]:
# nem sempre simples assim
tracks_df.to_csv('spotify_dataset.csv')

### SQLITE

In [58]:
# verificar os tipos dos dados
tracks_df.dtypes

track_id              object
track_name            object
track_duration_ms      int64
artist_name           object
album_name            object
album_release_date    object
popularity             int64
genre                 object
market                object
dtype: object

<p>A maioria dos dados está correta. A única alteração necessária é em 'album_release_date' para um objeto de data e hora.</p>

In [59]:
# o formato está misturado porque existem álbuns com ano-mês ou apenas o ano
tracks_df['album_release_date'] = pd.to_datetime(tracks_df['album_release_date'], format='mixed')

In [60]:
# verificar os tipos dos dados novamente
tracks_df.dtypes

track_id                      object
track_name                    object
track_duration_ms              int64
artist_name                   object
album_name                    object
album_release_date    datetime64[ns]
popularity                     int64
genre                         object
market                        object
dtype: object

In [61]:
# criar a tabela
create_table_spotify_tracks = """
    CREATE TABLE spotify_tracks(
        track_id                      TEXT,
        track_name                    TEXT,
        track_duration_ms              INT,
        artist_name                   TEXT,
        album_name                    TEXT,
        album_release_date        DATETIME,
        popularity                     INT,
        genre                         TEXT,
        market                        TEXT
    )
"""

# criar a conexão - caso o banco de dados não exista, este comando irá criá-lo
con = sqlite3.connect('spotify_tracks_db.sqlite')

# executar nossa query
con.execute(create_table_spotify_tracks)
con.commit()
con.close()

# inserir os dados no banco de dados
con = create_engine('sqlite:///spotify_tracks_db.sqlite')
tracks_df.to_sql('spotify_tracks', con=con, if_exists='append', index=False)

OperationalError: table spotify_tracks already exists

In [62]:
# consulta ao banco de dados
query = """
    SELECT * FROM spotify_tracks
"""

df_tracks_sqlite = pd.read_sql_query(query, con)

In [63]:
df_tracks_sqlite.head()

,track_id,track_name,track_duration_ms,artist_name,album_name,album_release_date,popularity,genre,market
0,38jy6kRlPt8z1GUS9WXeNh,93 Million Miles,216386,Jason Mraz,Love Is a Four Letter Word,2012-04-13 00:00:00.000000,69,acoustic,BR
1,3S0OXQeoh0w6AY8WQVckRW,I'm Yours,242946,Jason Mraz,We Sing. We Dance. We Steal Things. (Bonus Tra...,2008-05-01 00:00:00.000000,79,acoustic,BR
2,5vjLSffimiIP26QG5WcN2K,Hold On,198853,Chord Overstreet,Hold On,2017-02-03 00:00:00.000000,83,acoustic,BR
3,1EzrEOXmMH3G43AXT1y7pA,I'm Yours,242946,Jason Mraz,We Sing. We Dance. We Steal Things.,2008-05-12 00:00:00.000000,81,acoustic,BR
4,2qLMf6TuEC3ruGJg4SMMN6,Lucky,189613,Jason Mraz,We Sing. We Dance. We Steal Things. (Bonus Tra...,2008-05-01 00:00:00.000000,74,acoustic,BR


# REFERÊNCIAS

<p>Livros utilizados:</p>
<ul>
<li>Python para An&aacute;lise de Dados - 1&ordf; Edi&ccedil;&atilde;o</li>
<li>Data science do zero: primeiras regras com o Python - 1&ordf; Edi&ccedil;&atilde;o</li>
</ul>
<p>Links acessados:</p>
<ul>
<li>https://realpython.com/iterate-through-dictionary-python/</li>
<li>https://stackoverflow.com/questions/3294889/iterating-over-dictionaries-using-for-loops</li>
<li>https://www.geeksforgeeks.org/iterate-over-a-dictionary-in-python/</li>
<li>https://www.w3schools.com/python/python_dictionaries_loop.asp</li>
<li>https://medium.com/@vonagedev/3-ways-to-make-http-calls-with-python-requests-beginner-to-advanced-a5ca289ff342</li>
<li>https://requests.readthedocs.io/en/latest/</li>
<li>https://medium.com/reprogramabr/consumindo-a-api-do-spotify-um-breve-passo-a-passo-fd210312fdd</li>
<li>https://developer.spotify.com/documentation/web-api/tutorials/getting-started</li>
<li>https://medium.com/@maxtingle/getting-started-with-spotifys-api-spotipy-197c3dc6353b</li>
<li>https://medium.com/@alains/how-to-use-the-python-request-package-to-collect-data-from-apis-cfb4c5a6399a</li>
<li>https://matheusduzzi.medium.com/api-spotify-passo-a-passo-de-como-utilizar-e-poss%C3%ADveis-an%C3%A1lises-de-playlists-3984e0c46704</li>
<li>https://towardsdatascience.com/extracting-song-data-from-the-spotify-api-using-python-b1e79388d50?gi=135481c5c6a9</li>
</ul>